<a href="https://colab.research.google.com/github/SPS-L/pyeplan/blob/master/examples/1_bus_Planning_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Illustrative example for optimizing investment and operation of a single household

## Introduction

This example considers a single household and uses PyEPLAN to deside on the necessary generators that need to be installed to cover the load.

![Single Household](https://raw.githubusercontent.com/SPS-L/pyeplan/master/examples/1_bus.png)



This example is designed to be run on Google Colab. If this is the first time you use Colab, please take a 3-minute break and check out [this video](https://youtu.be/inN8seMm7UI).

To get a quickstart, you can simply click on "Runtime->Run all" and accept the prompt. This will execute the entire notebook and you can read through it like a report.

## Preparing the platform to execute the code

These commands make sure to remove any files remaining from previous executions.

In [ ]:
!rm -rf sample_data
!rm -rf 1_bus

The following command installs SVN and copies the project files from Github to Colab for execution.

In [ ]:
!apt-get -qq -y install subversion
!svn export https://github.com/SPS-L/pyeplan/trunk/examples/1_bus

The following command installs the necessary optimization solver for pyeplan to use.

In [ ]:
!apt-get -y -qq install glpk-utils

The following command installs PyEPLAN.

In [ ]:
!pip install -q pyeplan==0.4.3

This command is only used during debugging. To be removed at the final stage.

In [ ]:
!pip install git+https://github.com/SPS-L/pyeplan.git

## Defining the input data

The next step is to define the directory to the folder where the input data is located and define the reference node. The input data folder should consists of 'csv' files that contain data description of the load, newtork paramters and generation units as defined [here](https://pyeplan.sps-lab.org/user_guide/input.html#).

### Defining the total load demand at each hour

The total active power consumption at each of the three (3) hours for one (1) representative day is:

In [ ]:
import pandas as pd
pd.read_csv("1_bus/prep_dist.csv")

### Defining the input investment candidates

#### Wind turbine

In [ ]:
pd.read_csv("1_bus/cwin_dist.csv")

#### Solar PV

In [ ]:
pd.read_csv("1_bus/csol_dist.csv")

#### Diesel genset

In [ ]:
pd.read_csv("1_bus/cgen_dist.csv")

## Case studies
Case 1: No diesel candidates

Case 2: Solar + Wind + diesel candidated

Case 3: Total load reduced from 90 kW to 70 kW

## Using the invesment and operation planning module from PyEPLAN 

In [ ]:
import pyeplan as pye

The module is initialised with inputs including but not limited the following :
* The input directory for the data. The input data folder should consists of 'csv' files that contain data description of the load, newtork paramters and generation units as defined [here](https://pyeplan.sps-lab.org/user_guide/input.html#).
* ref_bus: Reference node
* dshed_cost: Demand Shedding Price
* rshed_cost: Renewable Shedding Price
* phase: Number of Phases (Default 3)
* sbase: Base Apparent Power (Default 1 kW)

In [ ]:
inosys = pye.inosys('1_bus', ref_bus = 0, dshed_cost = 100, rshed_cost = 0, phase = 3, )

## Solving the optimization problem

PyEPLAN can be used to solve the problem investment and operation planning problems simultaneously. In case no investment candidates are availble, a sole operation planning problem can be run by setting input 'onlyopr = True'. Available solvers inclde both open source solvers include glpk, cbc, and commercial solvers ipopt, gurobi given one has the required licences. If discrete capacities of investment units are available the input 'invest = True' this sets the investement-related decision variables to a binary nature where the unit capacity = nominal. Otherwise,  'invest = False' sets the investement-related decision variables to a continuous nature where the unit capacity <= nominal.

In [ ]:
inosys.solve(solver = 'glpk', onlyopr = False, invest = True, )

## Results

A folder named 'results' will be created with the output of the optimal solution to the planning problem. The different result files are defined [here](https://pyeplan.sps-lab.org/user_guide/output.html). Below we show the capital costs and operational costs obtained to satify the load in the 5-bus network.

### Total Investment and Operational Costs

In [ ]:
inosys.resCost()

### Number and capacity of wind units installed

The capacity and location of wind units installed is:

In [ ]:
inosys.resWind()

### Number and capacity of solar units installed

The capacity and location of solar units installed is:

In [ ]:
inosys.resSolar()

### Number and capacity of diesel units installed

The capacity and location of diesel units installed is:

In [ ]:
inosys.resConv()

### Amount of load curtailed

The level of load demand that has been curtailed at each node is:

In [ ]:
inosys.resCurt() 